<a href="https://colab.research.google.com/github/KhudV/graduation_paper_code/blob/main/text%20generation/%D0%9F%D1%80%D0%B5%D0%B4%D0%B2%D0%B0%D1%80%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0_SeqGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Обработка текста

In [ ]:
max_n_sent = 46
min_n_sent = 2

In [ ]:
import pandas
df = pandas.read_csv('/content/drive/MyDrive/ВКР/SeqGAN/datasets/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
import numpy as np
text = ' '.join(df['description'].values)
print('Всего символов:', len(text))

Всего символов: 1270877


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
text = text.lower()

In [ ]:
# количество упоминаний каждого слова (http://nlpx.net/archives/29)
from collections import Counter

cntr = Counter(nltk.word_tokenize(text)) # подсчет упоминаний

In [ ]:
print("Всего слов: ", len(list(cntr.most_common(10000000))))

Всего слов:  20659


In [ ]:
most_words = list(cntr.most_common(20659)) # наиболее частые N-1 слов (добавится заполнитель)
# most_words = cntr.elements() # вернуть слова

In [ ]:
print('Количество упоминаний самого редкого слова до обработки текста:', most_words[-1][0], most_words[-1][1])

Количество упоминаний самого редкого слова до обработки текста: roughneck 1


In [ ]:
# создание словаря слов
vocab = dict()
for i in range(len(most_words)):
  vocab[most_words[i][0]] = i
vocab['']=len(vocab) # добавить пробел (заполнитель)
vocab['_UNK_']=len(vocab) # добавить удаленное слово (удаляемые слова заменяются этим словом)
vocab['_START_']=len(vocab) # добавить стартовое слово в словарь

d = vocab
list_d = list(d.items())
list_d.sort(key=lambda i: i[1])

word = []
for i in range(len(list_d)):
  word.append(list_d[i][0])
print('Слов в словаре:', len(word))

Слов в словаре: 20662


In [ ]:
# разделение текста на предложения и слова (если в выборке есть знаки препинания, то примером считается предложение)
text_processed = []
for sent in nltk.sent_tokenize(text):
  text_processed.append(nltk.word_tokenize(sent))

# разделение текста на строки и слова (если в выборке нет знаков препинания или примеры разделены построчно, то примером считается строка)
#text_processed = []
#sets = text.split('\n') # -|-
#for i, sent in enumerate(sets):
#  if(i%(len(sets)//10)==0):
#    print(float(i)/float(len(sets)), '%')
#  text_processed.append(nltk.word_tokenize(sent))

In [ ]:
min_len = len(text_processed[0])
max_len = len(text_processed[0])

sr_len = 0
for i in range(len(text_processed)):
  sr_len += len(text_processed[i])
  min_len = min(min_len, len(text_processed[i]))
  max_len = max(max_len, len(text_processed[i]))

print("Предложений: ", len(text_processed))
print("Минимальная длина", min_len)
print("Средняя длина", sr_len/len(text_processed))
print("Максимальная длина", max_len)
print("Количество слов", sr_len)

Предложений:  9672
Минимальная длина 2
Средняя длина 24.122828784119108
Максимальная длина 65
Количество слов 233316


In [ ]:
# удаление коротких и длиных примеров
DEL_sentences=0
len_texts = len(text_processed)

i=0
ii=0
while i < len(text_processed):
  if(ii%(len_texts//100) == 0):
    print(ii//(len_texts//100), u'В обработке: ', len(text_processed), u"Удалено:", DEL_sentences, u'Сохранено: ', ii-DEL_sentences)
  
  if(len(text_processed[i]) < min_n_sent):# or len(text_processed[i]) > max_n_sent):#
    del(text_processed[i])
    DEL_sentences+=1
  else:
    text_processed[i] = text_processed[i][:max_n_sent]
    i+=1
  ii+=1
print('Предложений: ', len(text_processed)) #112138 #214986 #571377

0 В обработке:  9672 Удалено: 0 Сохранено:  0
1 В обработке:  9672 Удалено: 0 Сохранено:  96
2 В обработке:  9672 Удалено: 0 Сохранено:  192
3 В обработке:  9672 Удалено: 0 Сохранено:  288
4 В обработке:  9672 Удалено: 0 Сохранено:  384
5 В обработке:  9672 Удалено: 0 Сохранено:  480
6 В обработке:  9672 Удалено: 0 Сохранено:  576
7 В обработке:  9672 Удалено: 0 Сохранено:  672
8 В обработке:  9672 Удалено: 0 Сохранено:  768
9 В обработке:  9672 Удалено: 0 Сохранено:  864
10 В обработке:  9672 Удалено: 0 Сохранено:  960
11 В обработке:  9672 Удалено: 0 Сохранено:  1056
12 В обработке:  9672 Удалено: 0 Сохранено:  1152
13 В обработке:  9672 Удалено: 0 Сохранено:  1248
14 В обработке:  9672 Удалено: 0 Сохранено:  1344
15 В обработке:  9672 Удалено: 0 Сохранено:  1440
16 В обработке:  9672 Удалено: 0 Сохранено:  1536
17 В обработке:  9672 Удалено: 0 Сохранено:  1632
18 В обработке:  9672 Удалено: 0 Сохранено:  1728
19 В обработке:  9672 Удалено: 0 Сохранено:  1824
20 В обработке:  9672 Уд

In [ ]:
# замена не вошедших в словарь слов спец. словом и удаление предложений содержащих много замененных слов
DEL_sentences=0
len_texts = len(text_processed)
i=0
ii=0
while i < len(text_processed):
  if(ii%(len_texts//100) == 0):
    print(ii//(len_texts//100), u'В обработке: ', len(text_processed), u"Удалено:", DEL_sentences, u'Сохранено: ', ii-DEL_sentences)
  
  text_processed[i] = text_processed[i][:max_n_sent] # удаление лишней части предложения

  ln_n = len(text_processed[i])
  ln_s = 0
  j=0
  while j < ln_n:
    if(text_processed[i][j] not in word):
      #del(text_processed[i][j]) continue
      text_processed[i][j] = '_UNK_' #замена слов
      ln_s+=1
    j+=1
  if(ln_s>0):#ln_n*0.05):#если больше, чем N слов в примере не вошло в словарь, удалить пример
    del(text_processed[i])
    DEL_sentences+=1
  else:
    i+=1
  ii+=1
print("Удалено предложений (с недостающими словами):", DEL_sentences)
print('Осталось предложений: ', len(text_processed))

0 В обработке:  9672 Удалено: 0 Сохранено:  0
1 В обработке:  9672 Удалено: 0 Сохранено:  96
2 В обработке:  9672 Удалено: 0 Сохранено:  192
3 В обработке:  9672 Удалено: 0 Сохранено:  288
4 В обработке:  9672 Удалено: 0 Сохранено:  384
5 В обработке:  9672 Удалено: 0 Сохранено:  480
6 В обработке:  9672 Удалено: 0 Сохранено:  576
7 В обработке:  9672 Удалено: 0 Сохранено:  672
8 В обработке:  9672 Удалено: 0 Сохранено:  768
9 В обработке:  9672 Удалено: 0 Сохранено:  864
10 В обработке:  9672 Удалено: 0 Сохранено:  960
11 В обработке:  9672 Удалено: 0 Сохранено:  1056
12 В обработке:  9672 Удалено: 0 Сохранено:  1152
13 В обработке:  9672 Удалено: 0 Сохранено:  1248
14 В обработке:  9672 Удалено: 0 Сохранено:  1344
15 В обработке:  9672 Удалено: 0 Сохранено:  1440
16 В обработке:  9672 Удалено: 0 Сохранено:  1536
17 В обработке:  9672 Удалено: 0 Сохранено:  1632
18 В обработке:  9672 Удалено: 0 Сохранено:  1728
19 В обработке:  9672 Удалено: 0 Сохранено:  1824
20 В обработке:  9672 Уд

In [ ]:
# преобразование текста в набор чисел
for i in range(len(text_processed)):
  for j in range(len(text_processed[i])):
    text_processed[i][j] = vocab[text_processed[i][j]]

In [ ]:
# дополнение предложений до заданной длины заполнителем (' ' - пробелом)
max_n_sent = max_n_sent
for i in range(len(text_processed)):
  while len(text_processed[i]) < max_n_sent:
    text_processed[i].append(vocab[''])

In [ ]:
import pickle
obj = (word, vocab)
output = open('/content/drive/My Drive/ВКР/SeqGAN/datasets/netflix_titles_vocab.pkl', 'wb')
pickle.dump(obj, output, 2)
output.close()

In [ ]:
print('Предложений: ', len(text_processed)) #112138 #214986
for i in range(len(text_processed)):
  text_processed[i] = ' '.join(map(str, text_processed[i]))
# print('Предложений: ', len(text_processed)) #112138 #214986
# text_processed = '\n'.join(map(str, text_processed)) + '\n'
# print('Символов: ', len(text_processed)) #15224801 #29791310

Предложений:  9672


In [ ]:
print('\n'.join(map(str, text_processed[:15])) + '\n')

19 10 62 3628 3 491 6 8 27 2 468 10460 4870 1921 8 107 7 4135 5 4136 428 4 65 59 244 200 3 7457 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659
23 4871 442 28 0 352 2 0 4872 108 82 158 45 4 418 1106 0 7458 4137 154 20 10 221 24 241 1584 28 853 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659
4 326 8 33 18 0 245 184 569 2 1415 779 7459 5 8 640 87 6 2628 49 2879 30 0 363 5 327 4138 99 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659
7460 2 7461 5 4873 1002 181 100 659 3 2880 109 28 3 2069 364 730 7 31 2069 14 17 1679 214 48 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659
7 0 85 6 4139 2242 481 4 617 209 42 46 5912 10461 2070 2 12 4874 26 7462 129 5 8 44 353 2431 27 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20

In [ ]:
from random import shuffle
shuffle(text_processed)

print('Примеров: ', len(text_processed))

Примеров:  9672


In [ ]:
text_file = open(u"/content/drive/My Drive/ВКР/SeqGAN/datasets/netflix_titles_train.txt", "w")
text_file.write('\n'.join(map(str, text_processed[:int(len(text_processed)*0.9)])) + '\n')
text_file.close()

text_file = open(u"/content/drive/My Drive/ВКР/SeqGAN/datasets/netflix_titles_test.txt", "w")
text_file.write('\n'.join(map(str, text_processed[int(len(text_processed)*0.9):])) + '\n')
text_file.close()

In [ ]:
print(text_processed[:10])

['386 5534 12833 298 803 2 160 2 1045 5 5556 2 2778 4631 3865 9 65 18 8 2578 901 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659', '131 0 1484 109 2579 75 786 7 3 1095 6 19698 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659', '5740 3 1618 6 10 249 33 2 0 29 41 2106 19 12 937 168 129 5 1205 442 9 0 9556 685 154 28 58 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659', '17 432 1900 541 5 237 910 1155 18 180 3 37 2 19 1105 19 3 1945 5 1915 63 32 4459 1 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659 20659', '16 0 634 2719 124 13 0 528 197 16593 2 15 140 174 1614 1 20659 20659 20659 20659 20659 20659 20659 20

In [ ]:
print(vocab[''], vocab['i'], word[-1], word[20658])

20659 2470 _START_ roughneck
